In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
import csv
import tqdm

# Scraping

### env variables

In [3]:
BASE_PATH = '../data/pcs-scraping'
RESULTS_PATH = '../data/pcs-scraping/results/rider'
RANKINGS_PATH = '../data/pcs-scraping/pcs-rankings/rider'
TEAMS_PATH = '../data/pcs-scraping/teams/rider'
CALENDARS_PATH = '../data/pcs-scraping/calendars'
STARTLISTS_PATH = '../data/pcs-scraping/startlists'
RACERESULTS_PATH = '../data/pcs-scraping/race_results'

### Scraping functions

In [4]:
def get_rider_names(n_pages):
    
    rider_names = []

    offsets = np.arange(0, 3001, 100)

    for offset in offsets[:n_pages]:

        url = f'https://www.procyclingstats.com/rankings.php?date=2022-01-12&nation=&age=&zage=&page=smallerorequal&team=&offset={offset}&continent=&teamlevel=&filter=Filter&p=me&s=uci-individual'
        res = requests.get(url)

        tables = pd.read_html(res.content)
        rider_names.append(tables[0])

        time.sleep(0.5)
    
    return rider_names

def normalize_rider_name(rider_name):
    
    surname = rider_name.split(" ")[-1].lower()
    name = "-".join(rider_name.split(" ")[:-1]).lower()
    full_name = surname + '-' + name
    
    return full_name

def clean_pcs_table_results(df_table):
    
    df_table.drop('Unnamed: 3', axis=1, inplace=True)
    df_table.drop('Unnamed: 8', axis=1, inplace=True)
    df_table.rename(columns={'Unnamed: 2': 'GC'}, inplace=True)
    
    return df_table

def clean_pcs_table_ranking(df_table):
    
    df_table.rename(columns={'Unnamed: 0': 'year'}, inplace=True)

    return df_table

def get_rider_stats(rider_name, years):

    pcs_ranking = []
    results = []

    for year in years:

        try:
            url = f'https://www.procyclingstats.com/rider/{rider_name}/{year}'
            res = requests.get(url)

            tables = pd.read_html(res.content)
            pcs_ranking.append(clean_pcs_table_ranking(tables[1]))
            results.append(clean_pcs_table_results(tables[0]))

            time.sleep(0.5)
            
        except Exception as e:
            print(e)

    df_results = [(year, x) for year, x in zip(years, results) if not x.empty]
    
    return df_results, pcs_ranking[0]

def init_dirs():

    if not os.path.isdir(BASE_PATH):
        os.makedirs(BASE_PATH)
        
    if not os.path.isdir(RESULTS_PATH):
        os.makedirs(RESULTS_PATH)
        
    if not os.path.isdir(RANKINGS_PATH):
        os.makedirs(RANKINGS_PATH)
        
    if not os.path.isdir(TEAMS_PATH):
        os.makedirs(TEAMS_PATH)

def save_data(rider_name, results, pcs_ranking):
        
    # check if rider already has results data
    if not os.path.isdir(os.path.join(RESULTS_PATH, rider_name)):
        os.mkdir(os.path.join(RESULTS_PATH, rider_name))
    
    # check if rider already has pcs-ranking data
    if not os.path.isdir(os.path.join(RANKINGS_PATH, rider_name)):
        os.mkdir(os.path.join(RANKINGS_PATH, rider_name))
        
    # save season results
    #[x[1].to_csv(f'../data/pcs-scraping/results/rider/{rider_name}/{x[0]}.csv', index=False) for x in results]
    [x[1].to_csv(os.path.join(RESULTS_PATH, rider_name , f'{x[0]}.csv'), index=False) for x in results]
    
    # save pcs_ranking
    #pcs_ranking.to_csv(f'../data/pcs-scraping/pcs-ranking/rider/{rider_name}/pcs_ranking.csv', index=False)
    pcs_ranking.to_csv(os.path.join(RANKINGS_PATH, rider_name, 'pcs_ranking.csv'), index=False)

### Initialize data directories

In [ ]:
init_dirs()

### Get Rider Names

In [ ]:
rider_names = get_rider_names(n_pages=10)

In [ ]:
riders = []
[riders.extend(x['Rider']) for x in rider_names]

rider_names = [normalize_rider_name(x) for x in riders]

In [ ]:
with open(os.path.join(BASE_PATH, 'rider_names.csv'), 'w') as f:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(rider_names)

In [5]:
rider_names = list(pd.read_csv(os.path.join(BASE_PATH, 'rider_names.csv')))

### Get Race Results and Rankings

- NEED TO SCRAP ALL POSSIBLE YEARS (BEFORE 2011) !!!

- RIDER NAMES IN URL NOT CORRECT FOR FAILURES

In [6]:
years = np.arange(2000, 2023)

for rider_name in tqdm.tqdm(rider_names[:200]):

    try:
        results, pcs_ranking = get_rider_stats(rider_name, years)
        save_data(rider_name, results, pcs_ranking)
    except:
        print(rider_name)
        pass

  8%|▊         | 17/200 [06:02<1:05:08, 21.36s/it]

No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found


  9%|▉         | 18/200 [06:13<55:43, 18.37s/it]  

No tables found
jonas-vingegaard


 13%|█▎        | 26/200 [09:09<1:05:32, 22.60s/it]

...

 97%|█████████▋| 194/200 [1:05:41<02:05, 20.96s/it]

No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found
No tables found


 98%|█████████▊| 195/200 [1:05:46<01:21, 16.24s/it]

No tables found
juan-ayuso


100%|██████████| 200/200 [1:07:29<00:00, 20.25s/it]


In [8]:
rider_names_corrected_1 = ['jonas-vingegaard-rasmussen', 'mikkel-honore', 'aleksey-lutsenko', 'ben-o-connor',
                         'michael-valgren-andersen', 'biniam-girmay', 'miguel-angel-lopez', 'johan-esteban-chaves',
                         'michal-kwiatkowski', 'jesus-herrada-lopez', 'odd-christian-eiking', 'magnus-cort-nielsen',
                         'daniel-felipe-martinez', 'luis-leon-sanchez', 'tobias-halland-johannessen', 'jose-manuel-diaz-gallego',
                         'juan-ayuso-pesquera']
rider_names_corrected_2 = []

years = np.arange(2000, 2023)

for rider_name in tqdm.tqdm(rider_names_corrected_1):

    try:
        results, pcs_ranking = get_rider_stats(rider_name, years)
        save_data(rider_name, results, pcs_ranking)
    except:
        print(rider_name)
        pass

100%|██████████| 17/17 [06:02<00:00, 21.32s/it]


### Get Teams

In [9]:
def get_rider_teams(rider_name):
    
    time.sleep(0.5)
    
    url = f'https://www.procyclingstats.com/rider/{rider_name}'
    res = requests.get(url)
    
    soup = BeautifulSoup(res.content)
    
    possible_classes = ['list rdr-teams moblist moblist', 'list rdr-teams moblist', 'list rdr-teams moblist moblist ']
    ul = list(filter(None, [soup.find('ul', {'class': class_}) for class_ in possible_classes]))
    
    if ul:
        
        season = [x.find('div', {'class': 'season'}).text for x in ul[0].find_all('li')]
        team = [x.find('div', {'class': 'name'}).text for x in ul[0].find_all('li')]
        teams = pd.DataFrame({'season': season, 'team': team})
        
        return teams

    else:
        
        print(rider_name, 'No Teams scraped')

def save_data(rider_name, teams):
        
    # check if rider already has teams data
    if not os.path.isdir(os.path.join(TEAMS_PATH, rider_name)):
        os.mkdir(os.path.join(TEAMS_PATH, rider_name))
        
    # save teams data
    teams.to_csv(os.path.join(TEAMS_PATH, rider_name, 'teams.csv'), index=False)

In [10]:
rider_names = list(pd.read_csv(os.path.join(BASE_PATH, 'rider_names.csv')))

In [11]:
for rider_name in tqdm.tqdm(rider_names[:200]):
    
    try:
        teams = get_rider_teams(rider_name)
        save_data(rider_name, teams)
    except:
        print(rider_name)

  9%|▉         | 18/200 [00:16<02:30,  1.21it/s]

jonas-vingegaard No Teams scraped
jonas-vingegaard


 14%|█▎        | 27/200 [00:28<03:06,  1.08s/it]

frølich-honoré-mikkel No Teams scraped
frølich-honoré-mikkel


 20%|█▉        | 39/200 [00:44<02:58,  1.11s/it]

...

 91%|█████████ | 182/200 [03:12<00:15,  1.17it/s]

manuel-díaz-josé No Teams scraped
manuel-díaz-josé


 98%|█████████▊| 195/200 [03:24<00:04,  1.22it/s]

juan-ayuso No Teams scraped
juan-ayuso


100%|██████████| 200/200 [03:31<00:00,  1.06s/it]


In [12]:
for rider_name in tqdm.tqdm(rider_names_corrected):
    
    try:
        teams = get_rider_teams(rider_name)
        save_data(rider_name, teams)
    except:
        print(rider_name)

100%|██████████| 17/17 [00:17<00:00,  1.01s/it]


- RIDER NAMES NOT CORRECT ON URL FOR FAILURES

### Get race calendar

In [13]:
def get_race_calendar(years):

    calendar = []

    for year in years:

        try:
            url = f'https://www.procyclingstats.com/races.php?year={year}&circuit=&class=&filter=Filter'
            res = requests.get(url)

            tables = pd.read_html(res.content)
            calendar.append(tables[0].dropna())

            time.sleep(0.5)
            
        except Exception as e:
            print(e)

    df_calendar = [(year, cal) for year, cal in zip(years, calendar) if not cal.empty]
    
    return df_calendar

def save_data(calendar):
        
    if not os.path.isdir(CALENDARS_PATH):
        os.mkdir(CALENDARS_PATH)
        
    # save calendar data
    calendar[1].to_csv(os.path.join(CALENDARS_PATH, f'{calendar[0]}.csv'), index=False)

In [14]:
years = np.arange(2010, 2023)
calendar = get_race_calendar(years)
[save_data(cal) for cal in calendar]

[None, None, None, None, None, None, None, None, None, None, None, None, None]

### Get Race startlist + results

In [15]:
def convert_racename_to_url(racename):
    
    race_url = racename.values[0].lower().replace("'", "-").replace(" ", "-")
    
    return race_url
    
def get_race_results(race_url, year):
    
    url = f'https://www.procyclingstats.com/race/{race_url}/{year}'
    res = requests.get(url)

    tables = pd.read_html(res.content)
    last_stage = tables[0]
    gc = tables[1]
    points = tables[2]
    kom = tables[3]
    youth = tables[4]
    teams = tables[5]
    
    # ALSO NEED individual stages, kom, points, youth, teams final + stage results
    
    return last_stage, gc, points, kom, youth, teams

def get_startlist(race_url, year):
    
    url = f'https://www.procyclingstats.com/race/{race_url}/{year}/gc/startlist/alphabetical-with-filters'
    res = requests.get(url)
    
    startlist = pd.read_html(res.content)[0].drop(['Unnamed: 3'], axis=1)
    
    return startlist

def save_data(startlist, PATH, race_url, year):
        
    if not os.path.isdir(os.path.join(PATH, race_url, year)):
        os.makedirs(os.path.join(PATH, race_url, year))
        
    # save data
    startlist.to_csv(os.path.join(PATH, race_url, year, 'startlist.csv'), index=False)

### Get startlist of one race

In [31]:
year = '2021'
target_race = "Giro d'Italia"

calendar = pd.read_csv(os.path.join(CALENDARS_PATH, f'{year}.csv'))
race_url = convert_racename_to_url(calendar[calendar['Race'].str.contains(target_race)].Race)

startlist = get_startlist(race_url, year)
save_data(startlist, STARTLISTS_PATH, race_url, year)

### Get startlist of all races in year

In [39]:
year = '2021'
for race in calendar.Race:
    
    time.sleep(0.5)
    try:
        race_url = race.lower().replace(' ', '-')
        startlist = get_startlist(race_url, year)
        save_data(startlist, STARTLISTS_PATH, race_url, year)
    except:
        print(race)

New Zealand Cycle Classic
Vuelta al Tachira en Bicicleta
Gravel and Tar Classic
Gravel and Tar la Femme
Clàssica Comunitat Valenciana 1969 - Gran Premio Valencia
National Championships Qatar - ITT
National Championships Qatar - Road Race
Grand Prix Cycliste la Marseillaise
Etoile de Bessèges - Tour du Gard
National championships Australia - ITT
National championships Australia MJ - ITT
National championships Australia MU23 - ITT
National championships Australia WE - ITT
National championships Australia WJ - ITT
National Championships Australia WU23 - ITT
National championships Namibia - ITT
National championships Namibia MJ - ITT
National championships Namibia MU23 - ITT
National championships Namibia WE - ITT
National championships Namibia WJ - ITT
National championships Namibia WU23 - ITT
National championships Australia MJ - Road Race
National Championships Australia U23 - Road Race
National championships Australia WJ - Road Race
Grand Prix Alanya
National Championships Australia - 

National Championships Panama MJ - ITT
National Championships Panama MU23 - ITT
National Championships Panama WE - ITT
National Championships Panama WJ - ITT
National Championships Panama WU23 - ITT
National Championships Bulgaria WJ - ITT
National Championships Estonia MU23 - Road Race
National Championships Israel - ITT
National Championships Israel MJ - ITT
National Championships Israel MU23 - ITT
National Championships Israel WE - ITT
National Championships Israel WJ - ITT
National Championships Israel WU23 - ITT
National Championships Latvia MU23 - Road Race
National Championships Lithuania MU23 - Road Race
National Championships Mexico MJ - Road Race
National Championships Mexico WE - Road Race
National Championships Mexico WJ - Road Race
National Championships Belize - ITT
National Championships Belize MJ - ITT
National Championships Belize WE - ITT
National Championships Belize WJ - ITT
National Championships Bulgaria WJ - Road Race
National Championships El Salvador - ITT
Nati

### Get race results

In [40]:
def save_data(results, PATH, race_url, year):
        
    if not os.path.isdir(os.path.join(PATH, race_url, year)):
        os.makedirs(os.path.join(PATH, race_url, year))
        
    # save data
    [res.to_csv(os.path.join(PATH, race_url, year, f'{name}.csv'), index=False) for res, name in zip(results, ['last_stage', 'gc', 'points', 'kom', 'youth', 'team'])]

In [42]:
race_url = convert_racename_to_url(calendar[calendar['Race'].str.contains(target_race)].Race)
results = get_race_results(race_url, year)
save_data(results, RACERESULTS_PATH, race_url, year)

### Get race results of all races in year

In [43]:
year = '2021'
for race in calendar.Race:
    
    time.sleep(0.5)
    try:
        race_url = race.lower().replace(' ', '-')
        results = get_race_results(race_url, year)
        save_data(results, RACERESULTS_PATH, race_url, year)
    except:
        print(race)

New Zealand Cycle Classic
Vuelta al Tachira en Bicicleta
Gravel and Tar Classic
Gravel and Tar la Femme
Clàssica Comunitat Valenciana 1969 - Gran Premio Valencia
National Championships Qatar - ITT
National Championships Qatar - Road Race
Grand Prix Cycliste la Marseillaise
Etoile de Bessèges - Tour du Gard
National championships Australia - ITT
National championships Australia MJ - ITT
National championships Australia MU23 - ITT
National championships Australia WE - ITT
National championships Australia WJ - ITT
National Championships Australia WU23 - ITT
National championships Namibia - ITT
National championships Namibia MJ - ITT
National championships Namibia MU23 - ITT
National championships Namibia WE - ITT
National championships Namibia WJ - ITT
National championships Namibia WU23 - ITT
National championships Australia MJ - Road Race
National Championships Australia U23 - Road Race
National championships Australia WJ - Road Race
Grand Prix Alanya
National Championships Australia - 

National Championships Mexico MJ - ITT
National Championships Mexico MU23 - ITT
National Championships Mexico WE - ITT
National Championships Mexico WJ - ITT
National Championships Mexico WU23 - ITT
National Championships Panama - ITT
National Championships Panama MJ - ITT
National Championships Panama MU23 - ITT
National Championships Panama WE - ITT
National Championships Panama WJ - ITT
National Championships Panama WU23 - ITT
National Championships Bulgaria WJ - ITT
National Championships Estonia MU23 - Road Race
National Championships Israel - ITT
National Championships Israel MJ - ITT
National Championships Israel MU23 - ITT
National Championships Israel WE - ITT
National Championships Israel WJ - ITT
National Championships Israel WU23 - ITT
National Championships Latvia MU23 - Road Race
National Championships Lithuania MU23 - Road Race
National Championships Mexico MJ - Road Race
National Championships Mexico WE - Road Race
National Championships Mexico WJ - Road Race
Belgrade G